# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

In [47]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode

%matplotlib inline

# 設定 data_path
dir_data = '../../Lecture_data/'

In [50]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: ../../Lecture_data/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


In [40]:
"""
YOUR CODE HERE
"""
# 1: 計算 AMT_ANNUITY 的 q0 - q100
q_name = []
values = []
for i in np.arange(0, 101, 1):
    q_name.append('q' + str(i))
    temp = np.percentile(app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'], q = i)
    values.append(temp)
 
data = {'q': q_name, 'value': values}
df = pd.DataFrame(data)
print(df)

        q      value
0      q0    1615.50
1      q1    6182.91
2      q2    6750.00
3      q3    7875.00
4      q4    8703.00
5      q5    9000.00
6      q6    9000.00
7      q7    9553.50
8      q8   10125.00
9      q9   10503.00
10    q10   11074.50
11    q11   11430.00
12    q12   11970.00
13    q13   12375.00
14    q14   12838.50
15    q15   13302.00
16    q16   13500.00
17    q17   13500.00
18    q18   13896.00
19    q19   14350.50
20    q20   14701.50
21    q21   15124.50
22    q22   15583.50
23    q23   15970.50
24    q24   16209.00
25    q25   16524.00
26    q26   16852.50
27    q27   17109.00
28    q28   17487.00
29    q29   17806.50
..    ...        ...
71    q71   32458.50
72    q72   32895.00
73    q73   33376.50
74    q74   33984.00
75    q75   34596.00
76    q76   35345.16
77    q77   35806.50
78    q78   36328.50
79    q79   36747.00
80    q80   37516.50
81    q81   37948.50
82    q82   38556.00
83    q83   39456.00
84    q84   40135.50
85    q85   40806.00
86    q86   4

In [41]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

"""
Your Code Here
"""
q_50 = df.loc[50, 'value']
print(q_50)
app_train.loc[app_train['AMT_ANNUITY'].isnull(), 'AMT_ANNUITY'] = q_50

print("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
24903.0
After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [46]:
# 2.2 Normalize values to -1 to 1
print("== Original data range ==")
print(app_train['AMT_ANNUITY'].describe())

def normalize_value(x):
    temp = x - np.min(x)
    temp = temp/(np.max(x) - np.min(x)) - 0.5
    y = 2*temp 
    
    return y

app_train['AMT_ANNUITY_NORMALIZED'] = normalize_value(app_train['AMT_ANNUITY'])

print("\n== Normalized data range ==")
print(app_train['AMT_ANNUITY_NORMALIZED'].describe())

== Original data range ==
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64

== Normalized data range ==
count    307511.000000
mean         -0.801154
std           0.113049
min          -1.000000
25%          -0.883714
50%          -0.818357
75%          -0.742752
max           1.000000
Name: AMT_ANNUITY_NORMALIZED, dtype: float64


In [103]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

# 列出重複最多的數值
"""
Your Code Here
"""
value_most = mode(app_train[~app_train['AMT_GOODS_PRICE'].isnull()]['AMT_GOODS_PRICE'])
print('mode = %.1f ' %(value_most[0]))

mode_goods_price = list(app_train['AMT_GOODS_PRICE'].value_counts().index)
#print(app_train['AMT_GOODS_PRICE'].value_counts().index)


app_train['AMT_GOODS_PRICE_NEW'] = app_train['AMT_GOODS_PRICE']
app_train.loc[app_train['AMT_GOODS_PRICE_NEW'].isnull(), 'AMT_GOODS_PRICE_NEW'] = value_most[0]


print("After replace NAs, numbers of row that AMT_GOODS_PRICE_NEW is NAs: %i" % sum(app_train['AMT_GOODS_PRICE_NEW'].isnull()))

Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278
mode = 450000.0 
After replace NAs, numbers of row that AMT_GOODS_PRICE_NEW is NAs: 0


In [122]:
a = np.random.rand(10)
b = np.random.rand(10)

nan = [1, 3, 5, 7]
for i in nan:
    a[i] = np.nan
    
data = {'A':a, 'B': b}
test = pd.DataFrame(data)
print(test)

print('\n\n NaN - > 0 \n\n')

#test['A'].isnull()
test.loc[test['A'].isnull(), 'A'] = 0
print(test)

          A         B
0  0.149767  0.338894
1       NaN  0.844977
2  0.300114  0.186560
3       NaN  0.187914
4  0.328642  0.530914
5       NaN  0.776299
6  0.838591  0.164992
7       NaN  0.018936
8  0.526942  0.684985
9  0.629226  0.192471


 NaN - > 0 


          A         B
0  0.149767  0.338894
1  0.000000  0.844977
2  0.300114  0.186560
3  0.000000  0.187914
4  0.328642  0.530914
5  0.000000  0.776299
6  0.838591  0.164992
7  0.000000  0.018936
8  0.526942  0.684985
9  0.629226  0.192471
